In [1]:
using ForwardDiff
using PyPlot

In [2]:
function newtmin( obj, x0; maxIts=100, optTol=1e-6)
    # Newton method with Armijo backtracking and some treatment with Hessian, treatment 2
     # Minimize a function f using Newton’s method.
     # obj:  a function that evaluates the objective value,
     # gradient, and Hessian at a point x, i.e.,
     # (f, g, H) = obj(x)
     # x0: starting point.
     # maxIts (optional): maximum number of iterations.
     # optTol (optional): optimality tolerance based on
     #                    ||grad(x)|| <= optTol*||grad(x0)||
    mu=1e-4
    epsilon=0.01
    x=x0
    status = 0
    its = 0
    (f0,g0,H0)=obj(x0)
    (f,g,H)=obj(x)
    (V,S)=eig(H)
    bar=ones(size(H,1))*epsilon
    bar=Diagonal(bar)
    bH=S*max(bar,abs(Diagonal(V)))*S'
    while status != 1
        alpha = 1
        xnew = x-alpha*inv(bH)*g
        (fnew,gnew,Hnew)=obj(xnew)
        sts =-fnew+f-alpha*mu*g'*inv(bH)*g
        while sts[1]<0
            alpha=alpha/2
            xnew = x-alpha*inv(bH)*g
            (fnew,gnew,Hnew)=obj(xnew)
            sts=-fnew+f-alpha*mu*g'*inv(bH)*g
        end
        x = x-alpha*inv(bH)*g
        (f,g,H)=obj(x)
        (V,S)=eig(H)
        bar=ones(size(H,1))*epsilon
        bar=Diagonal(bar)
        bH=S*max(bar,abs(Diagonal(V)))*S'
        its = its+1
        if norm(g)<= optTol*norm(g0)
            status = 1
        end
        if its>maxIts
            status = 1
        end
    end
return (x, its)
end

newtmin (generic function with 1 method)

In [11]:
m=100
r=10
#W=abs(randn(m,m))
U0=abs(randn(r,m))
V0=abs(randn(r,m))
W=U0'*V0

100x100 Array{Float64,2}:
 13.0666   8.03724   9.60611   8.83956  …   8.73721   7.27037  7.96041
 10.8483   7.89709  11.9698    7.1105       7.90821   7.93317  6.64264
 12.0092   6.82576  13.9854    8.53769      7.7302    7.97564  6.36778
  9.98365  8.26696  11.5527    6.11571      6.71319  10.2316   7.00503
 10.8469   5.63212  11.018     9.34211      8.65459   5.32328  5.61544
 10.5864   7.1384   10.8294    9.87955  …   9.91214   6.83377  7.60036
  7.37918  4.24808   7.51673   7.78066      7.31399   5.03524  5.45415
 11.1055   5.71596  10.3932    9.8019       9.31916   5.95593  6.83752
 10.4824   5.69206  10.36      9.3495       8.18432   6.64989  6.67182
  9.40674  5.41065   9.85848   6.19402      6.06034   5.84679  4.98764
  9.12307  4.55358   5.70949   3.23884  …   6.39951   6.56046  4.97215
 10.4741   6.24692  11.3318    8.2635       6.66695   5.42306  5.66369
  5.82866  4.05195   5.34133   6.12587      6.72641   4.94822  4.63192
  ⋮                                     ⋱          

In [7]:
function f(x,W,m,r,lambda)
    U=reshape(x[1:m*r],r,m)
    V=reshape(x[m*r+1:2*m*r],r,m)
    y=sum((U'*V-W).^2)
    y=y-lambda*sum(log(x))
    return y
end
g(x,W,m,r,lambda)=ForwardDiff.gradient(x -> f(x,W,m,r,lambda))(x)
H(x,W,m,r,lambda)=ForwardDiff.hessian(x -> f(x,W,m,r,lambda))(x)

H (generic function with 1 method)

In [2]:
function my(W,m,r)
    x=abs(ones(2*m*r))
    #t=zeros(10)
    #s=zeros(10)
    #for i=1:5
        lambda=1/1000
        x0=x
        function obj(x)
            myf = f(x,W,m,r,lambda)  # objective value at x
            myg = g(x,W,m,r,lambda)  # gradient at x
            myH = H(x,W,m,r,lambda)   # Hessian at x
            return (myf,myg,myH)
        end
        (x,its)=newtmin( obj, x0; maxIts=10, optTol=1e-3)
    #U=reshape(x[1:m*r],r,m)
    #V=reshape(x[m*r+1:2*m*r],r,m)
     #   t[i]=sum((U-U0).^2)+sum((V-V0).^2)
      #  s[i]=sqrt(f(x,W,m,r,0))
    #end
    #return (t,s)
end

my (generic function with 1 method)

In [70]:
(x,its)=my(W,m,r)

([0.296843,0.296843,0.296843,0.449824,0.449824,0.449824,0.645334,0.645334,0.645334,0.692125  …  0.845671,0.238979,0.238979,0.238979,0.842251,0.842251,0.842251,1.3006,1.3006,1.3006],4)

In [71]:
f(x,W,m,r,0)

77.91059668133741

In [72]:
norm(g(x,W,m,r,1/1000))
U=reshape(x[1:m*r],r,m)
    V=reshape(x[m*r+1:2*m*r],r,m)
norm(U-U0)+norm(V-V0)

6.235465317248654

In [12]:
i=1:5
plot(i,t)

LoadError: LoadError: UndefVarError: t not defined
while loading In[12], in expression starting on line 2

#Algorithms for Non-negative Matrix Factorization

In [2]:
function nur1s(W,m,r,maxIts, optTol)
    #algorithm mentioned in Algorithms for non-negative matrix factorization with cost 
    #function ||W-U'V||, here assume W to be square matrix
    U=abs(randn(r,m)) #initial U
    V=abs(randn(r,m)) #initial V
    sts=0
    its=1
    while sts==0
        Vnew=V.*((U*W)./(U*U'*V))
        Unew=U.*((Vnew*W')./(Vnew*Vnew'*U))
        if abs(sum(W-Unew'*Vnew).^2-sum((W-U'*V).^2))<optTol
            sts=1
        end
        if its>=maxIts
            sts=1
        end
        V=Vnew
        U=Unew
        its=its+1
    end
    return (U,V,sum((W-U'*V).^2))
end

nur1s (generic function with 1 method)

In [22]:
@time (U,V,merror)=nur1s(W,m,r,1000, 1e-6)

elapsed time: 3.265463127 seconds (1765552824 bytes allocated, 41.10% gc time)


(
10x200 Array{Float64,2}:
 1.12024      0.518518     …  0.617873    0.166568    0.712606   
 2.05648      1.35729         1.34024     1.2846      1.329      
 0.0159769    0.657685        0.66437     0.00364119  1.21751    
 7.37825e-11  0.253384        2.22211     1.14559     0.471183   
 0.967511     1.03804         6.80357e-9  1.68131     0.443493   
 0.355467     1.91292      …  1.38654     0.0311517   0.549727   
 0.310636     0.373367        1.23675     0.808355    0.000249259
 3.88626e-6   1.99936e-6      0.0683145   1.2505      0.403016   
 0.975919     0.230324        0.892319    1.42377     0.625756   
 3.94664e-5   1.65279e-11     0.434732    0.798785    0.759419   ,

10x200 Array{Float64,2}:
 0.480974    0.531862  0.169649  …  1.13852      0.705556    0.887544  
 0.216752    0.304336  0.491363     0.188347     1.13883     0.830342  
 0.925531    0.8544    0.97708      0.88025      0.581158    0.62462   
 0.862481    0.959071  1.1433       1.32424      1.76241     0.0132945

In [29]:
function nur2s(W,m,r,maxIts, optTol)
    #algorithm mentioned in Algorithms for non-negative matrix factorization with cost 
    #function D(W||U'V), here assume W to be square matrix
    #first of all, define cost function D(W||U'V)
    function D(A,B)
        result = sum(A.*log(A./B)-A+B)
        return result
    end
    U=abs(randn(r,m)) #initial U
    V=abs(randn(r,m)) #initial V
    sts=0
    its=1
    while sts==0
        Vnew=V.*(U*(W./(U'*V)))./(sum(U,2)*ones(size(U,2))')
        Unew=U.*(Vnew*(W./(U'*Vnew))')./(sum(Vnew,2)*ones(size(Vnew,2))')
        if abs(D(W,Unew'*Vnew)-D(W,U'*V))<optTol
            sts=1
        end
        if its>=maxIts
            sts=1
        end
        V=Vnew
        U=Unew
        its=its+1
    end
    return (U,V,D(W,U'*V))
end

nur2s (generic function with 1 method)

In [30]:
(U,V,merror)=nur2s(W,m,r,1000, 1e-6)

(
3x70 Array{Float64,2}:
 0.825632  0.593242  0.533018  1.05059   …  0.839011  1.05051   0.343257
 1.17568   0.492925  1.23432   0.146368     0.835431  0.069183  0.104661
 1.53655   0.8029    0.520534  0.601086     1.07559   2.40312   0.253681,

3x70 Array{Float64,2}:
 2.01889   0.038569  0.995316  0.034537  …  1.19797   0.151912  1.45518
 0.379996  0.274988  0.730401  1.88538      1.31504   1.72654   1.57593
 0.786726  0.147184  0.560137  1.28918      0.461957  0.28637   1.79317,

0.003186740653090249)

#Algorithms and Applications for Approximate Nonnegative Matrix Factorization

In [4]:
function alss(W,m,r,maxIts, optTol)
    #basic alternating least squares algorithm for nonegative matrix factorization, here first
    #assume W to be a square matrix
    tole=sum(W.^2)*optTol
    #initialize U 
    U=abs(randn(r,m))
    V=abs(randn(r,m))
    #our main iteration
    its=0
    while (its<maxIts && sum((W-U'*V).^2)>tole)
        #least square
        Vnew=pinv(U*U')*(U*W)
        #set all negative elements in V to be 0
        Vnew=max(Vnew,0)
        #least square
        Unew=pinv(Vnew*Vnew')*(Vnew*W')
        #set all negative elements in U to be 0
        Unew=max(Unew,0)
        U=Unew
        V=Vnew
        its=its+1
    end
    return (its,U,V,sum((W-U'*V).^2))
end

alss (generic function with 1 method)

In [ ]:
@time alss(W,m,r,10000, 1e-4)

#Non-negative Matrix Factorization Based On Alternating Non-negativity Constrained Least Squares and Active Set Method

In [2]:
function nls(B,y)
    #this algorithm compute the solution for min_{g>=0}||Bg-y||_2 by Active Set Method, while
    #y is m*1 dimension
    
    #pre calculate B^TB and B^Ty
    btb=B'*B
    bty=B'*y
    #////////////////////////////
    (m,n)=size(B)
    g=zeros(n)
    eps=ones(n) #indicate active set of g
    w=bty-btb*g
    sts=0 #indicate when the iteration terminate
    while (sts==0)
        tep=zeros(size(w))
        tep[eps.==1]=w[eps.==1]
        t=indmax(tep)
        eps[t]=0
        z=zeros(n)
        tepl=btb[eps.==0,eps.==0]
        z[eps.==0]=pinv(tepl)*bty[eps.==0] #one can do some 
        #improvement here by pre-calculate B^TB and B^Ty
        ssts=0# indicate when the small loop terminate
        if (sum(z[eps.==0].<=0)==0)
            ssts=1
        end
        while (ssts==0)
            value=ones(n)/0
            value[(z.<=0)&(eps.==0)]=g[(z.<=0)&(eps.==0)]./(g-z)[(z.<=0)&(eps.==0)]
            q=indmin(value)
            alpha=g[q]/(g[q]-z[q])
            g=g+alpha*(z-g)
            eps[(eps.==0)&(g.==0)]=1
            z=zeros(n)
            tepl=btb[eps.==0,eps.==0]
            z[eps.==0]=pinv(tepl)*bty[eps.==0] #one can do some 
            #improvement here by pre-calculate B^TB and B^Ty
            if (sum(z[eps.==0].<=0)==0)
                ssts=1
            end
        end
        g=z
        w=bty-btb*g
        if sum(eps)==0
            sts=1
        end
        if (sum(w[eps.==1].>0)==0)
            sts=1
        end
    end
    return g
end

nls (generic function with 1 method)

In [3]:
function anlss(W,m,r,maxIts, optTol)
    #ANLS method for square matrix
    tole=sum(W.^2)*optTol
    U=abs(randn(r,m))
    #U=ones(r,m)
    V=zeros(r,m)
    sts=0
    its=1
    while sts==0
        for j=1:m
            V[:,j]=nls(U',W[:,j])
        end
        for j=1:m
            U[:,j]=nls(V',W'[:,j])
        end
        if its>maxIts
            sts=1
        end
        if sum((W-U'*V).^2)<tole
            sts=1
        end
        its=its+1
    end
    return(its,U,V,sum((W-U'*V).^2))
end

anlss (generic function with 1 method)

In [ ]:
@time (its,U,V,terror)=anlss(W,m,r,1000, 1e-4)

#Fast algorithm for the solution of large-scale non-negativity-constrained least squares problems

In [2]:
function fcnnls(C, A)
    # NNLS using normal equations and the fast combinatorial strategy 
    #    
    # I/O: [K, Pset] = fcnnls(C, A);
    # K = fcnnls(C, A); 
    #
    # C is the nObs   lVar coefficient matrix
    # A is the nObs   pRHS matrix of observations
    # K is the lVar   pRHS solution matrix
    # Pset is the lVar   pRHS passive set logical array 
    #
    # M. H. Van Benthem and M. R. Keenan
    # Sandia National Laboratories
    
    # Pset: set of passive sets, one for each column
    # Fset: set of column indices for solutions that have not yet converged
    # Hset: set of column indices for currently infeasible solutions
    # Jset: working set of column indices for currently optimal solutions
    
    # Check the input arguments for consistency and initialize error(nargchk(2,2,nargin))
    (nObs, lVar) = size(C)
    if size(A,1) != nObs
        error("C and A have imcompatible sizes")
    end 
    pRHS = size(A,2)
    W = zeros(lVar, pRHS)
    iter = 0
    maxiter = 3*lVar
    # Precompute parts of pseudoinverse
    CtC = C'*C
    CtA = C'*A
    # Obtain the initial feasible solution and corresponding passive set 
    
    K = CtC\CtA
    Pset = K.> 0
    K[~Pset] = 0
    D = K+1-1
    Fset = find(~all(Pset,1))
    
    # Active set algorithm for NNLS main loop 
    while ~isempty(Fset)
        # Solve for the passive variables (uses subroutine below) 
        K[:,Fset] = cssls(CtC, CtA[:,Fset], Pset[:,Fset])
        # Find any infeasible solutions
        Hset = Fset[find(any(K[:,Fset] .<0,1))]'
        # Make infeasible solutions feasible (standard NNLS inner loop) 
        if ~isempty(Hset)
            nHset = length(Hset)
            alpha = zeros(lVar, nHset)
            while ~isempty(Hset) & (iter < maxiter)
                iter = iter + 1
                alpha[:,1:nHset] = Inf
                # Find indices of negative variables in passive set 
                (i, j) = findn(Pset[:,Hset'[:,1]]  & (K[:,Hset'[:,1]] .< 0))
                hIdx = sub2ind((lVar, nHset), i, j)'
                negIdx = sub2ind(size(K), i, Hset[j])'
                alpha[hIdx] = D[negIdx]./(D[negIdx] - K[negIdx])
                (alphaMin,minIdx) = findmin(alpha[:,1:nHset],1)
                (minIdx,temp)=ind2sub(size(alpha),minIdx'[:,1])
                minIdx=minIdx'
                alpha[:,1:nHset] = repmat(alphaMin, lVar, 1)
                D[:,Hset'[:,1]] = D[:,Hset'[:,1]]-alpha[:,1:nHset].*(D[:,Hset'[:,1]]-K[:,Hset'[:,1]])
                idx2zero = sub2ind(size(D), minIdx'[:,1], Hset'[:,1])'
                D[idx2zero] = 0
                Pset[idx2zero] = false
                K[:, Hset] = cssls(CtC, CtA[:,Hset'[:,1]], Pset[:,Hset'[:,1]]);
                Hset = find(any(K .< 0,1))' 
                nHset = length(Hset)
            end 
        end
#\\\\
        #Make sure the solution has converged
        if iter == maxiter
            error("Maximum number iterations exceeded")
        end 
        #Check solutions for optimality
        W[:,Fset] = CtA[:,Fset]-CtC*K[:,Fset]
        Jset = find(all(~Pset[:,Fset].*W[:,Fset] .<= 0,1))#!!!!
        Fset = setdiff(Fset, Fset[Jset])
        #For non-optimal solutions, add the appropriate variable to Pset
        if ~isempty(Fset)
            (mx,mxidx)=findmax(~Pset[:,Fset].*W[:,Fset],1)
            temp=Pset[:,Fset]
            temp[mxidx]= true
            Pset[:,Fset]=temp
            D[:,Fset] = K[:,Fset]
        end 
    end
    return K
end
# ****************************** Subroutine ****************************
function cssls(CtC, CtA, Pset)
    #Solve the set of equations CtA = CtC*K for the variables in set Pset 
    # using the fast combinatorial approach
    K = zeros(size(CtA))
    if ( isempty(Pset) || all(Pset))
        K = CtC\CtA 
    else 
        (lVar,pRHS) = size(Pset)
        codedPset = (2.^collect(lVar-1:-1:0))'*Pset ##########!!!!!!!!!!!!!!!!!!!
        sortedPset=sort(codedPset'[:,1])'
        sortedEset=sortperm(codedPset'[:,1])'
        breaks = diff(sortedPset,2)
        breakIdx = [0 find(breaks)' pRHS]
        for k = 1:length(breakIdx)-1
            cols2solve = sortedEset[breakIdx[k]+1:breakIdx[k+1]];
            vars = Pset[:,sortedEset[breakIdx[k]+1]]; 
            K[vars,cols2solve] = CtC[vars,vars]\CtA[vars,cols2solve];
        end 
    end
    return K
end

cssls (generic function with 1 method)

In [3]:
function anls(W,m,r,maxIts, optTol)
    #ANLS method equiped with fcnnls
    tole=sum(W.^2)*optTol
    U=abs(randn(r,m))
    #U=ones(r,m)
    V=zeros(r,m)
    sts=0
    its=1
    while sts==0
        V=fcnnls(U',W)
        U=fcnnls(V',W')
        if its>maxIts
            sts=1
        end
        if sum((W-U'*V).^2)<tole
            sts=1
        end
        its=its+1
    end
    return(its,U,V,sum((W-U'*V).^2))
end

anls (generic function with 1 method)

In [13]:
@time (its,U,V,terror)=anls(W,m,r,1000, 1e-4)

elapsed time: 0.098042308 seconds (34910192 bytes allocated)


(40,
10x100 Array{Float64,2}:
 0.558214  0.859772   0.217363  0.0       …  0.740038   0.461194  0.49029 
 1.81487   3.95124    1.89479   3.94155      0.0        2.49515   0.322788
 0.206698  0.7889     1.23891   0.0          0.974904   1.95096   1.83823 
 0.662146  0.290036   0.789895  0.845956     0.0958468  0.530639  0.556953
 4.75888   1.10539    0.532685  0.0          3.23496    1.75853   1.39399 
 3.5853    1.45609    0.616731  1.59582   …  2.36595    2.29313   0.604146
 2.32892   1.95035    2.44067   3.05074      0.478831   0.586314  1.38404 
 1.60687   3.28131    3.20607   6.26806      2.80594    2.7734    0.313507
 0.466501  1.27139    2.52692   0.595294     0.29624    0.699912  0.0     
 0.267583  0.0909513  0.136117  0.0          0.719412   1.56228   0.898769,

10x100 Array{Float64,2}:
 1.94398   0.927864  1.15215     1.75564   …  2.48787   0.714519  0.562935
 0.361459  0.68612   0.839196    0.0          0.0       0.419166  0.332179
 0.525796  0.471596  2.12327     2.4918    

In [1]:
using Convex
using ECOS

Base.MethodError(f=Base.VersionNumber, args=("2.0.2",))


In [4]:
function mynls(B,W)
m=size(W,2)
r=size(B,1)
V=Variable(r,m)
problem = minimize(sumsquares((W-B'*V)), [V >= 0])
solve!(problem)
problem.status # :Optimal, :Infeasible, :Unbounded etc.
problem.optval
    return V.value
end

mynls (generic function with 1 method)

In [28]:
function myanlss(W,m,r,maxIts, optTol)
    #ANLS method for square matrix
    tole=sum(W.^2)*optTol
    U=abs(randn(r,m))
    #U=ones(r,m)
    V=zeros(r,m)
    sts=0
    its=1
    while sts==0
        V=mynls(U,W)
        U=mynls(V,W')
        if its>maxIts
            sts=1
        end
        if sum((W-U'*V).^2)<tole
            sts=1
        end
        its=its+1
    end
    return(its,U,V,sum((W-U'*V).^2))
end

myanlss (generic function with 1 method)

In [26]:

V=mynls(U,W);
        U=mynls(V,W');
sum((W-U'*V).^2)



ECOS 2.0.2 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.322e+00  +1e+03  1e+00  9e-01  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  -1.038e-01  -1.497e-01  +1e+02  3e-02  3e-01  1e-01  1e-01  0.9253  2e-02   1  1  1 |  0  0
 2  -5.349e-02  -5.469e-02  +4e+01  2e-02  2e-01  9e-02  4e-02  0.7263  2e-01   1  1  1 |  0  0
 3  -4.410e-02  -6.460e-02  +4e+01  2e-02  2e-02  5e-02  4e-02  0.0567  9e-01   2  2  3 |  0  0
 4  -5.752e-02  -6.874e-02  +4e+01  2e-02  2e-02  8e-02  4e-02  0.1628  7e-01   3  2  3 |  0  0
 5  -3.353e-02  -4.909e-02  +3e+01  1e-02  4e-03  5e-02  3e-02  0.6297  5e-01   3  3  3 |  0  0
 6  -1.531e-02  -2.528e-02  +1e+01  6e-03  2e-03  2e-02  1e-02  0.6799  2e-01   2  2  2 |  0  0
 7  -1.226e-02  -2.197e-02  +1e+01  6e-03  2e-03  1e-02  1e-02  0.5648  6e-01   3  3  3 |  0  0
 8  -5.766e-03  -1.127e-02  +5e+00  3e-03  6e-

 2  2 |  0  0
64  +4.441e+01  +4.441e+01  +1e-09  8e-07  2e-14  4e-09  1e-12  0.8625  3e-01   0  1  1 |  0  0
65  +4.441e+01  +4.441e+01  +2e-09  6e-07  3e-14  5e-09  2e-12  0.2609  8e-01   1  2  2 |  0  0
66  +4.441e+01  +4.441e+01  +8e-10  6e-07  8e-15  3e-09  8e-13  0.9683  3e-01   0  1  1 |  0  0
67  +4.441e+01  +4.441e+01  +9e-10  6e-07  7e-14  3e-09  9e-13  0.1692  5e-01   1  2  2 |  0  0
68  +4.441e+01  +4.441e+01  +5e-10  5e-07  2e-14  2e-09  5e-13  0.8887  3e-01   0  1  1 |  0  0
69  +4.441e+01  +4.441e+01  +9e-10  4e-07  2e-14  2e-09  9e-13  0.2064  8e-01   1  2  2 |  0  0
70  +4.441e+01  +4.441e+01  +4e-10  4e-07  8e-15  2e-09  4e-13  0.9531  3e-01   0  1  1 |  0  0
71  +4.441e+01  +4.441e+01  +5e-10  4e-07  5e-14  2e-09  5e-13  0.1921  6e-01   1  2  2 |  0  0
72  +4.441e+01  +4.441e+01  +3e-10  4e-07  7e-15  1e-09  3e-13  0.8245  3e-01   0  1  1 |  0  0
73  +4.441e+01  +4.441e+01  +6e-10  2e-07  1e-14  1e-09  6e-13  0.4594  7e-01   1  2  2 |  0  0
74  +4.441e+01  +4.441e+01

44.41184221477347

In [13]:
sum(W.^2)*1e-4

46.576181089368696

In [30]:
tole

0.002260171808471379

In [29]:
myanlss(W,m,r,1000, 1e-4)


ECOS 2.0.2 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.322e+00  +1e+03  1e+00  9e-01  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  -1.043e-01  -1.573e-01  +1e+02  4e-02  3e-01  1e-01  1e-01  0.9161  2e-02   1  1  1 |  0  0
 2  -1.577e-01  -1.588e-01  +9e+01  7e-02  5e-01  3e-01  9e-02  0.5142  5e-01   2  2  2 |  0  0
 3  -1.139e-01  -2.673e-01  +9e+01  3e-02  9e-02  2e-03  9e-02  0.1749  8e-01   1  1  1 |  0  0
 4  -1.684e-01  -3.915e-01  +7e+01  6e-02  3e-02  2e-02  7e-02  0.3291  5e-01   2  2  2 |  0  0
 5  -9.519e-02  -2.151e-01  +7e+01  3e-02  1e-02  1e-02  7e-02  0.1214  8e-01   2  2  3 |  0  0
 6  -1.958e-01  -4.278e-01  +6e+01  1e-01  7e-03  1e-01  6e-02  0.4583  6e-01   3  3  3 |  0  0
 7  +7.404e-03  -5.005e-02  +2e+01  2e-01  3e-03  2e-01  2e-02  0.6474  9e-02   3  2  3 |  0  0
 8  +4.219e-01  +3.914e-01  +2e+01  2e-01  3e-

 0.7484  1e-01   1  1  1 |  0  0
35  +4.513e+03  +4.513e+03  +3e-05  4e-05  2e-07  5e-03  3e-08  0.5230  4e-01   1  2  2 |  0  0
36  +4.517e+03  +4.517e+03  +1e-05  3e-05  7e-08  2e-03  1e-08  0.7307  2e-01   1  1  1 |  0  0
37  +4.517e+03  +4.517e+03  +1e-05  2e-05  5e-08  2e-03  1e-08  0.4305  5e-01   1  2  2 |  0  0
38  +4.519e+03  +4.519e+03  +5e-06  2e-05  2e-08  8e-04  5e-09  0.8263  2e-01   1  1  1 |  0  0
39  +4.519e+03  +4.519e+03  +4e-06  1e-05  1e-08  6e-04  4e-09  0.7319  5e-01   1  2  2 |  0  0
40  +4.520e+03  +4.520e+03  +1e-06  1e-05  3e-09  2e-04  1e-09  0.8405  1e-01   1  1  1 |  0  0
41  +4.520e+03  +4.520e+03  +1e-06  1e-05  3e-09  2e-04  1e-09  0.3377  5e-01   1  2  2 |  0  0
42  +4.521e+03  +4.521e+03  +6e-07  9e-06  1e-09  1e-04  6e-10  0.9267  4e-01   0  1  1 |  0  0
43  +4.521e+03  +4.521e+03  +6e-07  5e-06  7e-10  8e-05  6e-10  0.8886  5e-01   1  2  2 |  0  0
44  +4.521e+03  +4.521e+03  +3e-07  5e-06  3e-10  4e-05  3e-10  0.6702  1e-01   0  1  1 |  0  0
45  +4.

  +1e-08  8e-06  3e-12  3e-06  1e-11  0.5686  5e-01   0  1  1 |  0  0
63  +2.975e+03  +2.975e+03  +2e-09  1e-05  1e-12  1e-06  2e-12  0.9524  1e-01   2  1  1 |  0  0
64  +2.975e+03  +2.975e+03  +3e-09  1e-05  1e-12  1e-06  3e-12  0.1061  7e-01   3  1  1 |  0  0
65  +2.975e+03  +2.975e+03  +2e-09  9e-06  2e-12  7e-07  2e-12  0.6446  3e-01   0  1  1 |  0  0
66  +2.975e+03  +2.975e+03  +3e-09  8e-06  2e-12  8e-07  3e-12  0.0613  8e-01   0  1  1 |  0  0
67  +2.975e+03  +2.975e+03  +3e-09  7e-06  2e-12  9e-07  3e-12  0.9890  4e-01   0  1  1 |  0  0
68  +2.975e+03  +2.975e+03  +6e-09  8e-06  6e-13  1e-06  6e-12  0.5751  5e-01   3  1  1 |  0  0
69  +2.975e+03  +2.975e+03  +8e-10  8e-06  2e-12  5e-07  8e-13  0.9890  1e-01   0  1  1 |  0  0
70  +2.975e+03  +2.975e+03  +2e-09  7e-06  1e-12  4e-07  2e-12  0.3102  7e-01   0  1  1 |  0  0
71  +2.975e+03  +2.975e+03  +2e-09  8e-06  1e-12  1e-06  2e-12  0.8524  7e-01   0  1  1 |  0  0
72  +2.975e+03  +2.975e+03  +3e-09  8e-06  1e-12  1e-06  3e-12  0.

1  1 |  0  0
49  +2.223e+03  +2.223e+03  +1e-07  4e-06  4e-11  1e-05  1e-10  0.5009  6e-01   1  2  2 |  0  0
50  +2.223e+03  +2.223e+03  +6e-08  3e-06  1e-11  5e-06  6e-11  0.7459  2e-01   0  1  1 |  0  0
51  +2.223e+03  +2.223e+03  +7e-08  3e-06  1e-11  5e-06  7e-11  0.4437  7e-01   1  2  2 |  0  0
52  +2.223e+03  +2.223e+03  +3e-08  3e-06  5e-12  3e-06  3e-11  0.8272  3e-01   0  1  1 |  0  0
53  +2.223e+03  +2.223e+03  +4e-08  1e-06  3e-12  2e-06  4e-11  0.7815  5e-01   1  2  2 |  0  0
54  +2.223e+03  +2.223e+03  +1e-08  1e-06  1e-12  1e-06  1e-11  0.9169  2e-01   0  1  1 |  0  0
55  +2.223e+03  +2.223e+03  +1e-08  1e-06  1e-12  1e-06  1e-11  0.2615  4e-01   1  2  2 |  0  0
56  +2.223e+03  +2.223e+03  +7e-09  1e-06  4e-13  6e-07  7e-12  0.7867  3e-01   0  1  1 |  0  0
57  +2.223e+03  +2.223e+03  +1e-08  7e-07  4e-13  5e-07  1e-11  0.7920  6e-01   1  2  2 |  0  0
58  +2.223e+03  +2.223e+03  +2e-09  7e-07  1e-13  2e-07  2e-12  0.9410  1e-01   0  1  1 |  0  0
59  +2.223e+03  +2.223e+03 

14  5e-08  5e-13  0.1660  7e-01   0  1  1 |  0  0
68  +1.575e+03  +1.575e+03  +3e-10  3e-06  8e-14  5e-08  3e-13  0.9428  3e-01   0  1  1 |  0  0
69  +1.575e+03  +1.575e+03  +4e-10  2e-06  9e-14  6e-08  4e-13  0.0871  8e-01   0  1  1 |  0  0
70  +1.575e+03  +1.575e+03  +3e-10  2e-06  9e-14  4e-08  3e-13  0.8608  3e-01   0  1  1 |  0  0
71  +1.575e+03  +1.575e+03  +5e-10  2e-06  9e-14  5e-08  5e-13  0.1365  8e-01   1  2  2 |  0  0
72  +1.575e+03  +1.575e+03  +3e-10  2e-06  1e-14  6e-08  3e-13  0.9890  3e-01   3  1  1 |  0  0
73  +1.575e+03  +1.575e+03  +5e-10  2e-06  8e-14  5e-08  5e-13  0.4091  5e-01   0  1  1 |  0  0
74  +1.575e+03  +1.575e+03  +3e-10  2e-06  5e-14  4e-08  3e-13  0.5956  2e-01   0  1  1 |  0  0
75  +1.575e+03  +1.575e+03  +3e-10  2e-06  8e-14  4e-08  3e-13  0.0704  8e-01   0  1  1 |  0  0
76  +1.575e+03  +1.575e+03  +2e-10  1e-06  6e-14  3e-08  2e-13  0.8233  2e-01   0  1  1 |  0  0
77  +1.575e+03  +1.575e+03  +4e-10  1e-06  6e-14  3e-08  4e-13  0.1829  8e-01   1  2  

...

-07  6e-15  2e-10  6e-14  0.2525  7e-01   1  2  2 |  0  0
100  +5.600e+01  +5.600e+01  +2e-11  2e-07  4e-16  1e-10  2e-14  0.8935  2e-01   0  1  1 |  0  0
Maximum number of iterations reached, stopping.

Close to OPTIMAL (within feastol=1.8e-07, reltol=3.9e-13, abstol=2.2e-11).
Runtime: 1.034647 seconds.


ECOS 2.0.2 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.322e+00  +1e+03  1e+00  9e-01  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  -1.058e-01  -1.520e-01  +1e+02  4e-02  3e-01  1e-01  1e-01  0.9225  2e-02   1  1  1 |  0  0
 2  -8.836e-02  -9.645e-02  +6e+01  3e-02  3e-01  1e-01  6e-02  0.7129  3e-01   2  2  2 |  0  0
 3  -6.186e-02  -1.267e-01  +5e+01  2e-02  8e-02  3e-02  5e-02  0.3774  7e-01   1  1  1 |  0  0
 4  -7.991e-02  -1.554e-01  +5e+01  3e-02  6e-02  6e-02  5e-02  0.2238  7e-01   2  2  2 |  0  0
 5  -4.239e-02  -8.465e-02 

 0  0
38  +5.201e+01  +5.201e+01  +4e-07  1e-05  1e-11  2e-06  4e-10  0.6956  3e-01   0  1  1 |  0  0
39  +5.202e+01  +5.202e+01  +3e-07  1e-05  6e-12  2e-06  3e-10  0.9890  5e-01   0  1  1 |  0  0
40  +5.202e+01  +5.202e+01  +4e-08  8e-06  6e-13  2e-07  4e-11  0.9890  6e-02   0  1  1 |  0  0
41  +5.202e+01  +5.202e+01  +7e-08  7e-06  5e-13  2e-07  7e-11  0.5399  6e-01   0  1  1 |  0  0
42  +5.202e+01  +5.202e+01  +1e-07  7e-06  4e-13  5e-07  1e-10  0.9890  8e-01   0  1  1 |  0  0
43  +5.203e+01  +5.203e+01  +1e-07  6e-06  3e-13  6e-07  1e-10  0.9890  5e-01   0  1  1 |  0  0
44  +5.203e+01  +5.203e+01  +1e-08  5e-06  2e-13  6e-08  1e-11  0.9890  6e-02   0  1  1 |  0  0
45  +5.203e+01  +5.203e+01  +2e-08  4e-06  1e-13  4e-08  2e-11  0.9890  3e-01   0  1  1 |  0  0
46  +5.203e+01  +5.203e+01  +6e-09  3e-06  6e-14  2e-08  6e-12  0.9890  1e-01   0  1  1 |  0  0
47  +5.203e+01  +5.203e+01  +7e-09  3e-06  4e-14  1e-08  7e-12  0.9890  2e-01   0  1  1 |  0  0
48  +5.203e+01  +5.203e+01  +3e-09

13  0.7074  2e-01   0  1  1 |  0  0
88  +4.833e+01  +4.833e+01  +2e-10  4e-07  1e-14  5e-10  2e-13  0.1657  7e-01   1  2  2 |  0  0
89  +4.833e+01  +4.833e+01  +1e-10  4e-07  3e-15  4e-10  1e-13  0.8123  2e-01   0  1  1 |  0  0
90  +4.833e+01  +4.833e+01  +2e-10  3e-07  2e-14  5e-10  2e-13  0.1856  7e-01   1  2  2 |  0  0
91  +4.833e+01  +4.833e+01  +8e-11  3e-07  8e-16  3e-10  8e-14  0.7713  2e-01   0  1  1 |  0  0
92  +4.833e+01  +4.833e+01  +1e-10  3e-07  1e-14  4e-10  1e-13  0.1722  7e-01   1  2  2 |  0  0
93  +4.833e+01  +4.833e+01  +6e-11  3e-07  2e-15  3e-10  6e-14  0.7761  2e-01   0  1  1 |  0  0
94  +4.833e+01  +4.833e+01  +1e-10  3e-07  1e-14  3e-10  1e-13  0.2071  7e-01   1  2  2 |  0  0
95  +4.833e+01  +4.833e+01  +5e-11  3e-07  4e-16  2e-10  5e-14  0.7871  2e-01   0  1  1 |  0  0
96  +4.833e+01  +4.833e+01  +7e-11  2e-07  1e-14  2e-10  7e-14  0.1575  7e-01   1  2  2 |  0  0
97  +4.833e+01  +4.833e+01  +4e-11  2e-07  2e-15  2e-10  4e-14  0.8192  3e-01   0  1  1 |  0  0
98  

3  4e-07  6e-11  0.1469  8e-01   1  2  2 |  0  0
61  +4.490e+01  +4.490e+01  +2e-08  4e-06  1e-13  2e-07  2e-11  0.9890  4e-01   0  1  1 |  0  0
62  +4.490e+01  +4.490e+01  +3e-08  3e-06  2e-13  1e-07  3e-11  0.6250  4e-01   0  1  1 |  0  0
63  +4.490e+01  +4.490e+01  +1e-08  3e-06  1e-13  5e-08  1e-11  0.7244  9e-02   0  1  1 |  0  0
64  +4.490e+01  +4.490e+01  +1e-08  3e-06  1e-13  5e-08  1e-11  0.1065  7e-01   0  1  1 |  0  0
65  +4.490e+01  +4.490e+01  +7e-09  3e-06  6e-14  3e-08  7e-12  0.7160  2e-01   0  1  1 |  0  0
66  +4.490e+01  +4.490e+01  +1e-08  3e-06  1e-13  3e-08  1e-11  0.1697  7e-01   1  2  2 |  0  0
67  +4.490e+01  +4.490e+01  +5e-09  2e-06  4e-14  2e-08  5e-12  0.7909  2e-01   0  1  1 |  0  0
68  +4.490e+01  +4.490e+01  +7e-09  2e-06  9e-14  2e-08  7e-12  0.1847  7e-01   1  2  2 |  0  0
69  +4.490e+01  +4.490e+01  +4e-09  2e-06  3e-14  1e-08  4e-12  0.7679  2e-01   0  1  1 |  0  0
70  +4.490e+01  +4.490e+01  +5e-09  2e-06  1e-13  2e-08  5e-12  0.1773  7e-01   1  2  2

(30,
10x100 Array{Float64,2}:
  1.5645       1.77722    0.78145    0.182135     …  0.784463     1.10065    
  1.06931      0.546151   0.463553   0.168905        0.426406     2.11691e-12
  2.09841      1.37001    2.73412    1.94118         1.79342e-12  2.06338    
  1.34549      1.59342    0.654047   0.84629         2.57416      0.629805   
 -2.09528e-12  0.0322549  0.720858   0.295048        5.77242e-12  1.45443    
  1.12797      0.866935   0.648999   0.717348     …  1.35594      1.45998    
  0.569566     0.718526   0.832803   0.980234        2.9922e-12   0.394077   
  3.07679      1.93073    0.460457   2.53013e-12     1.05676      2.34166    
  0.898169     2.22456    0.707962   0.311879        1.77338      1.02593    
  0.727308     1.11938    0.0915186  1.18837         2.13842      0.64198    ,

10x100 Array{Float64,2}:
 0.427107     0.624621     0.458212  …  0.0777885  1.26334      0.834471   
 0.0461107    0.373397     0.730265     0.806364   0.43444      0.0972273  
 0.197264  

In [11]:
IPython.Cell.options_default.cm_config.lineNumbers = true;

LoadError: LoadError: UndefVarError: IPython not defined
while loading In[11], in expression starting on line 1